##Ite wiki  

b2b-kentän ohjelmistoyritykset, it-palvelutarjoajat, web- ja mobiilikehittäjät sekä digitalisaation osaajayritykset.

In [ ]:
test = False

In [ ]:
%sh 
/databricks/python/bin/pip install --upgrade pip
/databricks/python/bin/pip install bs4

Collecting pip
 Downloading pip-21.0.1-py3-none-any.whl (1.5 MB)
Installing collected packages: pip
 Attempting uninstall: pip
 Found existing installation: pip 20.2.4
 Uninstalling pip-20.2.4:
 Successfully uninstalled pip-20.2.4
Successfully installed pip-21.0.1
Collecting bs4
 Downloading bs4-0.0.1.tar.gz (1.1 kB)
Collecting beautifulsoup4
 Downloading beautifulsoup4-4.9.3-py3-none-any.whl (115 kB)
Collecting soupsieve>1.2
 Downloading soupsieve-2.2.1-py3-none-any.whl (33 kB)
Building wheels for collected packages: bs4
 Building wheel for bs4 (setup.py): started
 Building wheel for bs4 (setup.py): finished with status 'done'
 Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=cad0f0e7e7d1dc8f9f0cda315617bce649220dec90417889fdd0dc44ef4ac323
 Stored in directory: /root/.cache/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4
Installing collected packages: soupsieve, beautifulsoup4, bs4
Successfully installed beautifulsoup4-4.9.3 bs4-0.0.1 soupsieve-2.2.1

In [ ]:
# Modules
from bs4 import BeautifulSoup
import datetime, requests, re
import pandas as pd, numpy as np
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from urllib.parse import quote

## Main table

Muoto: Yritys itewikin metadatoineen per rivi.

In [ ]:
# Function - Get company names as a list
def get_itewiki_companies() -> list:
    
    '''
    Retrieves Itewiki's list of companies from
    https://www.itewiki.fi/lista-yrityksista
    Returns: 
        Itewiki company names as a list
    
    '''
     
    url =  "https://www.itewiki.fi/lista-yrityksista"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser") 
               
    companies = ['https://www.itewiki.fi'+ str(webbi.get('href')) \
                 for link in soup.find_all('div', {'class':'col-sm-4'}) \
                 for webbi in link.find_all("a", class_="name")]
    
    return companies

In [ ]:
# Get company list
companies = get_itewiki_companies()
len(companies)

Out[5]: 2005

In [ ]:
companies[:5]

Out[6]: ['https://www.itewiki.fi/11latoa',
 'https://www.itewiki.fi/1up',
 'https://www.itewiki.fi/24apps',
 'https://www.itewiki.fi/2ndchain',
 'https://www.itewiki.fi/2ns-second-nature-security']

In [ ]:
companies[-5:]

Out[7]: ['https://www.itewiki.fi/zoneatlas',
 'https://www.itewiki.fi/zoner',
 'https://www.itewiki.fi/zscaler',
 'https://www.itewiki.fi/kompozure',
 'https://www.itewiki.fi/osterbottens-databyra']

In [ ]:
# Function -  Get company data
def get_itewiki_company_data(companies: list) -> pd.DataFrame:
    
    '''
    
    Retrieves Itewiki's company details:
        
         - Y-tunnus / Business ID
         - nimi / Company name
         - Kotisivu / Homepage
         - Perustamisvuosi / Year of establishment
         - Työntekijöiden lkm / Number of employees
         - Osaamisalueet / Knowledge area: 
             - Liiketoimintaprosessi / Business process
             - Erikoisosaaminen / Special skills
             - Teknologia / Technology
             - Toimialakokemus / Industry domain
             - Tarjonnan tyyppi / Supply
             - Omat tagit / Own tags
             
    Args: 
        Itewiki's list of companies listed at https://www.itewiki.fi/lista-yrityksista         
    
    Returns: 
        Pandas dataframe df
             
    ''' 

    # Initialize pandas dataframe
    index = pd.RangeIndex(start = 0, step = 1)    
    df = pd.DataFrame(index = index)

    # Retry configuration
    session = requests.Session()
    retries = Retry(connect = 4, backoff_factor = 0.3)
    adapter = HTTPAdapter(max_retries = retries)
    session.mount('https://', adapter)
    
    
    for index, element in enumerate(companies):
        
        if (index % 50 == 0):
          print('Batch ' + str(index) + '/' + str(len(companies)))
        
        r = session.get(element)            
        soup = BeautifulSoup(r.text, "html.parser")

        ### Yritysesittely:
        # - Y-tunnus
        # - Nimi
        # - Www
        # - Perustamisvuosi
        # - Työntekijöiden lkm
        
        try:
            workers = [td.string for td in soup.find("table", {"class": "table1"}).find_all("td")]
            
        except AttributeError:
            workers = ["" for i in range(6)]
        
        try:
            # Y-tunnus oikein:
            if bool(re.match(pattern=r'\d{7}-\d{1}', string=workers[3].string.strip())):
                df.loc[index, 'y_tunnus'] = workers[3].string.strip()
            # Y-tunnuksesta puuttu -:
            elif bool(re.match(pattern=r'\d*$', string=workers[3].string.strip()) and len(workers[3].string.strip())==8):    
                df.loc[index, 'y_tunnus'] = workers[3].string.strip()[:7] + '-' + workers[3].string.strip()[-1]
            # Y-tunnuksesta puuttu ensimmäinen 0 ja -:
            elif bool(re.match(pattern=r'\d*$', string=workers[3].string.strip()) and len(workers[3].string.strip())==7):     
                df.loc[index, 'y_tunnus'] = '0' + workers[3].string.strip()[:6] + '-' + workers[3].string.strip()[-1]
            # Muut virheelliset sellaisenaan (0,5% luokkaa koko aineistosta):
            else:
                df.loc[index, 'y_tunnus'] = workers[3].string.strip()
            
            # Nimi    
            df.loc[index, 'nimi'] = re.sub(';','', re.sub('https://www.itewiki.fi/', '', companies[index]))
            # www
            df.loc[index, 'url'] = soup.find("table", {"class": "table2"}).find("a").get("href").strip()
            # Perustamisvuosi
            df.loc[index, 'perustamisvuosi'] = re.sub('-', '', workers[-1].string.strip()) 
            # Työntekijöiden lukumäärä
            df.loc[index, 'tyontekijat_lkm'] = re.sub('-', '', workers[1].string.strip())
        
        except BaseException:
            continue
                            
        ### Osaamisalueet: 
        # - Liiketoimintaprosessi(osaaminen)
        # - Erikoisosaaminen
        # - Teknologia 
        # - Toimialakokemus
        # - Tarjonnan tyyppi
        # ja ko. osaamisalueen sisältävien referenssien lukumäärä.
        # - Omat tagit.
        
        # Until 2020
        #try:
        #    descr = [re.sub(';','', p_element.contents[-1].strip()) \
        #                for p_element in soup.find('h3').find_next_siblings() \
        #                if p_element.name=="p"]
                    
        #    df.loc[index, 'liiketoimintaprosessi'] = re.sub(';', '', descr[0])
        #    df.loc[index, 'erikoisosaaminen'] = re.sub(';', '', descr[1])
        #    df.loc[index, 'teknologia'] = re.sub(';', '', descr[2])
        #    df.loc[index, 'toimialakokemus'] = re.sub(';', '', descr[3])
        #    df.loc[index, 'tarjonnan_tyyppi'] = re.sub(';', '', descr[4])
        #    df.loc[index, 'omat_tagit'] = re.sub(';', '', descr[5])
        
        #except BaseException:
        #    continue
        
        osaamisalueet = ['Liiketoimintaprosessiosaaminen', 'Erikoisosaaminen', 
                         'Teknologia', 'Toimialakokemus', 'Tarjonnan tyyppi']
        for osaamisalue in osaamisalueet:
          
          try:
            osaamiset = '; '.join([re.sub('[\\n\\t()\\d+]', '', td_element.get_text()).strip() \
                                   for td_element in soup.find('h3', string = osaamisalue).find_next_sibling().find_all("td", class_="td_text")]) 
            df.loc[index, osaamisalue.lower()] = osaamiset
          
          # Ei ko. osaamisaluetta portfoliossa.
          except AttributeError:
            df.loc[index, osaamisalue.lower()] = ''
          
          try:
            referenssit = '; '.join([re.sub('[\\n\\t()\\D]', '', td_element.get_text()).strip() \
                                     for td_element in soup.find('h3', string = osaamisalue).find_next_sibling().find_all("td", class_="td_text")])
            df.loc[index, osaamisalue.lower() + '_referenssit'] = referenssit
          
          # Ei ko. osaamisaluetta portfoliossa.
          except AttributeError:
            df.loc[index, osaamisalue.lower() + '_referenssit'] = ''
            
        try:
          df.loc[index, 'omat_tagit'] = '; '.join([re.sub('[\\n\\t()\\d+]', '', td_element.get_text()).strip() \
                                                   for td_element in \
                                                   soup.find('h3', string = 'Omat tagit').find_next_sibling().find_all("div", class_="label label-default")])
        # Ei omia tageja portfoliossa.
        except AttributeError:
          df.loc[index, 'omat_tagit'] = ''
        
    df.columns = ['y_tunnus', 'nimi', 'url', 'perustamisvuosi', 'tyontekijat_lkm',
                  'liiketoimintaprosessi', 'liiketoimintaprosessi_refs_lkm',
                  'erikoisosaaminen', 'erikoisosaaminen_refs_lkm', 
                  'teknologia', 'teknologia_refs_lkm',
                  'toimialakokemus', 'toimialakokemus_refs_lkm', 
                  'tarjonnan_tyyppi', 'tarjonnan_tyyppi_refs_lkm',
                  'omat_tagit']
    return df

In [ ]:
# Get company data
data = get_itewiki_company_data(companies)

Batch 0/2005
Batch 50/2005
Batch 100/2005
Batch 150/2005
Batch 200/2005
Batch 250/2005
Batch 300/2005
Batch 350/2005
Batch 400/2005
Batch 450/2005
Batch 500/2005
Batch 550/2005
Batch 600/2005
Batch 650/2005
Batch 700/2005
Batch 750/2005
Batch 800/2005
Batch 850/2005
Batch 900/2005
Batch 950/2005
Batch 1000/2005
Batch 1050/2005
Batch 1100/2005
Batch 1150/2005
Batch 1200/2005
Batch 1250/2005
Batch 1300/2005
Batch 1350/2005
Batch 1400/2005
Batch 1450/2005
Batch 1500/2005
Batch 1550/2005
Batch 1600/2005
Batch 1650/2005
Batch 1700/2005
Batch 1750/2005
Batch 1800/2005
Batch 1850/2005
Batch 1900/2005
Batch 1950/2005
Batch 2000/2005

In [ ]:
data.shape

Out[10]: (2001, 16)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2001 entries, 0 to 2004
Data columns (total 16 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 y_tunnus 2001 non-null object
 1 nimi 2001 non-null object
 2 url 1986 non-null object
 3 perustamisvuosi 1986 non-null object
 4 tyontekijat_lkm 1986 non-null object
 5 liiketoimintaprosessi 1986 non-null object
 6 liiketoimintaprosessi_refs_lkm 1986 non-null object
 7 erikoisosaaminen 1986 non-null object
 8 erikoisosaaminen_refs_lkm 1986 non-null object
 9 teknologia 1986 non-null object
 10 teknologia_refs_lkm 1986 non-null object
 11 toimialakokemus 1986 non-null object
 12 toimialakokemus_refs_lkm 1986 non-null object
 13 tarjonnan_tyyppi 1986 non-null object
 14 tarjonnan_tyyppi_refs_lkm 1986 non-null object
 15 omat_tagit 1986 non-null object
dtypes: object(16)
memory usage: 345.8+ KB

In [ ]:
data.head(5)

Out[12]:

,y_tunnus,nimi,url,perustamisvuosi,tyontekijat_lkm,liiketoimintaprosessi,liiketoimintaprosessi_refs_lkm,erikoisosaaminen,erikoisosaaminen_refs_lkm,teknologia,teknologia_refs_lkm,toimialakokemus,toimialakokemus_refs_lkm,tarjonnan_tyyppi,tarjonnan_tyyppi_refs_lkm,omat_tagit
0,2454224-2,11latoa,http://11latoa.com,2012,1,Tuotekehitys ja suunnittelu,0,Mobiilikehitys; Ohjelmistokehitys; Webkehitys,0; 0; 0,Android; Open source; Windows Phone,0; 0; 0,Asiantuntijapalvelut,0,Konsultointi; Valmisohjelmisto,0; 0,
1,2707548-3,1up,http://www.oneuphigher.com,2016,200,HR; Tuotanto,0; 0,Mobiilikehitys; Ohjelmistokehitys; Pilvipalvel...,0; 0; 0; 0; 0,Android; Google; Julkaisujärjestelmä CMS; Linu...,0; 0; 0; 0; 0; 0; 0; 0,IT,0,Toteutustyö; Vuokratyövoima,0; 0,
2,2826596-7,24apps,http://www.24apps.online,2017,2,Asiakaspalvelu; Toiminnanohjaus ERP,0; 0,,,Microsoft,0,IT,0,Konsultointi; Tuki- ja ylläpitotyö,0; 0,
3,2532870-7,2ndchain,http://www.2ndchain.com,2013,,Logistiikka; Supply chain management SCM; Hank...,1; 1; 0; 0,Verkkokaupparatkaisut,1,,,,,,,Tukkukauppa; Verkkokaupan logistiikka
4,2324099-8,2ns-second-nature-security,http://www.2ns.fi,2010,30,"Laatu, turvallisuus ja ympäristö; Tietohallint...",0; 0; 0,Auditointi; Tietoturva,0; 0,Linux; Microsoft; Windows,0; 0; 0,"Asiantuntijapalvelut; Hotelli, matkailu ja rav...",0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0,Konsultointi; Koulutus,0; 0,Tietoturva-auditoinnit; hallinnollinen tietotu...


In [ ]:
data.tail(5)

Out[13]:

,y_tunnus,nimi,url,perustamisvuosi,tyontekijat_lkm,liiketoimintaprosessi,liiketoimintaprosessi_refs_lkm,erikoisosaaminen,erikoisosaaminen_refs_lkm,teknologia,teknologia_refs_lkm,toimialakokemus,toimialakokemus_refs_lkm,tarjonnan_tyyppi,tarjonnan_tyyppi_refs_lkm,omat_tagit
2000,2163580-6,zoneatlas,https://zoneatlas.com/,2008,,"Asiakaspalvelu; Laatu, turvallisuus ja ympäris...",0; 0; 0; 0; 0; 0,Graafinen suunnittelu; Integraatiot; Ketterät ...,0; 0; 0; 0; 0; 0; 0; 0; 0; 0,Amazon Web Services; Android; Azure; Javascrip...,0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0,"Asiantuntijapalvelut; Hotelli, matkailu ja rav...",0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0,Konsultointi; Koulutus; Toteutustyö; Tuki- ja ...,0; 0; 0; 0; 0,
2001,1985221-1,zoner,https://www.zoner.fi/,2005,20,Tietohallinto; Viestintä,0; 0,Palvelin- ja kapasiteettipalvelut; Pilvipalvel...,0; 0,Julkaisujärjestelmä CMS; Linux; Open source; P...,1; 1; 0; 0; 0,,,Tuki- ja ylläpitotyö,0,sähköposti; virtuaalipalvelin; Varmuuskopioint...
2002,3158148-5,zscaler,https://www.zscaler.com,2008,2500,Tietohallinto,0,IoT; It-infra loppukäyttäjäpalvelut; Pilvipalv...,0; 0; 0; 0; 0,Amazon Web Services; Azure; Google,0; 0; 0,,,,,Zero Trust; SASE
2003,2416123-9,kompozure,https://zure.com,2011,37,Myynti; Markkinointi; Toiminnanohjaus ERP; Pro...,4; 1; 1; 0; 0; 0,Arkkitehtuuri; Integraatiot; Ketterät menetelm...,5; 5; 5; 5; 4; 1; 1; 1; 0,C#; Microsoft; React; Azure,5; 5; 2; 0,Valmistava teollisuus; IT; Kauppa; Media; Pros...,2; 1; 1; 1; 1; 1; 0; 0,Konsultointi; Toteutustyö; Tuki- ja ylläpitoty...,5; 5; 5; 0,Microsoft Azure; IoT; Cloud; Data platform; Di...
2004,2680634-4,osterbottens-databyra,http://www.odb.fi,2015,3,Markkinointi; Projektinhallinta; Tuotekehitys ...,0; 0; 0,Graafinen suunnittelu; It-omaisuuden hallinta;...,0; 0; 0; 0,,,IT,0,Konsultointi; Toteutustyö,0; 0,


In [ ]:
# Post process data
for item in data.columns:
  data.loc[data[item] == '', item] = np.nan
  data.loc[data[item] == '-', item] = np.nan
            
data = data.dropna(axis = 0, how = 'all')
data = data[data.y_tunnus.notnull()]
data = data.drop(data[data.nimi == 'poistettu'].index)

# Add batch timestamp
#data['batch'] = pd.Series(time.strftime('%Y-%m-%d'), index = data.index)
timestamp = datetime.datetime.now().strftime("%Y-%m-%d")# %H:%M:%S")
data['batch'] = pd.Series(timestamp, index = data.index)

In [ ]:
data.shape

Out[15]: (1955, 17)

In [ ]:
data.head(5)

Out[16]:

,y_tunnus,nimi,url,perustamisvuosi,tyontekijat_lkm,liiketoimintaprosessi,liiketoimintaprosessi_refs_lkm,erikoisosaaminen,erikoisosaaminen_refs_lkm,teknologia,teknologia_refs_lkm,toimialakokemus,toimialakokemus_refs_lkm,tarjonnan_tyyppi,tarjonnan_tyyppi_refs_lkm,omat_tagit,batch
0,2454224-2,11latoa,http://11latoa.com,2012,1,Tuotekehitys ja suunnittelu,0,Mobiilikehitys; Ohjelmistokehitys; Webkehitys,0; 0; 0,Android; Open source; Windows Phone,0; 0; 0,Asiantuntijapalvelut,0,Konsultointi; Valmisohjelmisto,0; 0,NaN,2021-04-16
1,2707548-3,1up,http://www.oneuphigher.com,2016,200,HR; Tuotanto,0; 0,Mobiilikehitys; Ohjelmistokehitys; Pilvipalvel...,0; 0; 0; 0; 0,Android; Google; Julkaisujärjestelmä CMS; Linu...,0; 0; 0; 0; 0; 0; 0; 0,IT,0,Toteutustyö; Vuokratyövoima,0; 0,NaN,2021-04-16
2,2826596-7,24apps,http://www.24apps.online,2017,2,Asiakaspalvelu; Toiminnanohjaus ERP,0; 0,NaN,NaN,Microsoft,0,IT,0,Konsultointi; Tuki- ja ylläpitotyö,0; 0,NaN,2021-04-16
3,2532870-7,2ndchain,http://www.2ndchain.com,2013,NaN,Logistiikka; Supply chain management SCM; Hank...,1; 1; 0; 0,Verkkokaupparatkaisut,1,NaN,NaN,NaN,NaN,NaN,NaN,Tukkukauppa; Verkkokaupan logistiikka,2021-04-16
4,2324099-8,2ns-second-nature-security,http://www.2ns.fi,2010,30,"Laatu, turvallisuus ja ympäristö; Tietohallint...",0; 0; 0,Auditointi; Tietoturva,0; 0,Linux; Microsoft; Windows,0; 0; 0,"Asiantuntijapalvelut; Hotelli, matkailu ja rav...",0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0,Konsultointi; Koulutus,0; 0,Tietoturva-auditoinnit; hallinnollinen tietotu...,2021-04-16


In [ ]:
# Duplicate value -> Y-tunnus cannot be set as a Primary Key
data[data.y_tunnus=="2247943-8"]

Out[17]:

,y_tunnus,nimi,url,perustamisvuosi,tyontekijat_lkm,liiketoimintaprosessi,liiketoimintaprosessi_refs_lkm,erikoisosaaminen,erikoisosaaminen_refs_lkm,teknologia,teknologia_refs_lkm,toimialakokemus,toimialakokemus_refs_lkm,tarjonnan_tyyppi,tarjonnan_tyyppi_refs_lkm,omat_tagit,batch
617,2247943-8,finceptum,https://finceptum.fi/,2009,15,Tietohallinto,0,It-infrapalvelut; Konesalipalvelut; Käyttäjäha...,0; 0; 0; 0,Linux; SAP,0; 0,Asiantuntijapalvelut; IT,1; 1,Konsultointi; Valmisohjelmisto; Koulutus,1; 1; 0,identiteetinhallinta; idm; NetIQ; OpenStack; p...,2021-04-16
1126,2247943-8,micro-focus-suomi,https://www.microfocussuomi.fi,2009,15,Tietohallinto,2,Käyttäjähallinta; Tietoturva; IoT,1; 1; 0,NaN,NaN,"Julkishallinto; IT; Kuljetus, liikenne ja logi...",2; 1; 1; 0,Konsultointi; Valmisohjelmisto; Koulutus,4; 4; 1,NetIQ; lokien hallinta; SIEM; idm; pääsynhallinta,2021-04-16


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1955 entries, 0 to 2004
Data columns (total 17 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 y_tunnus 1955 non-null object
 1 nimi 1955 non-null object
 2 url 1940 non-null object
 3 perustamisvuosi 1652 non-null object
 4 tyontekijat_lkm 1571 non-null object
 5 liiketoimintaprosessi 1621 non-null object
 6 liiketoimintaprosessi_refs_lkm 1621 non-null object
 7 erikoisosaaminen 1639 non-null object
 8 erikoisosaaminen_refs_lkm 1639 non-null object
 9 teknologia 1241 non-null object
 10 teknologia_refs_lkm 1241 non-null object
 11 toimialakokemus 1536 non-null object
 12 toimialakokemus_refs_lkm 1536 non-null object
 13 tarjonnan_tyyppi 1697 non-null object
 14 tarjonnan_tyyppi_refs_lkm 1697 non-null object
 15 omat_tagit 1212 non-null object
 16 batch 1955 non-null object
dtypes: object(17)
memory usage: 354.9+ KB

### Extra tables

Liiketoimintaprosessit, erikoisosaamiset, teknologiat, toimialakokemukset ja tarjonnan tyypit omina tauluinaan.

Muoto: Prosessi, erikoisosaaminen, teknologia jne. per rivi kullekin Y-tunnukselle.

In [ ]:
# Explodes pandas dataframe string entry to separate rows
def explode(data, lst_cols, fill_value = '', preserve_index = False):
    
    '''
    
    Multiple list columns - all list columns must have the same # of elements in each row:

           y_tunnus          text           num
      0   1234567-8        aa; bb; cc     1; 3; 2
      1   2345678-9        dd; ee         0; 4
      
     explode(df, ['text','num'], fill_value = '')

           y_tunnus     text    num
      0   1234567-8      aa      1
      1   1234567-8      bb      3
      2   1234567-8      cc      2
      3   2345678-9      dd      0
      4   2345678-9      ee      4
      
    '''
    
    idx_cols = data.columns.difference(lst_cols)
    # Calculate lengths of lists
    lens = data[lst_cols[0]].str.len()
    # Preserve original index values    
    idx = np.repeat(data.index.values, lens)
    # Create "exploded" dataframe
    res = pd.DataFrame({col:np.repeat(data[col].values, lens) \
                        for col in idx_cols}, index = idx).assign(**{col:np.concatenate(data.loc[lens>0, col].values) \
                                                                     for col in lst_cols})
    # Revert the original index order
    res = res.sort_index()
    # Reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    
    return res

In [ ]:
# Create word space table for the given 'column'.

def itewiki_wordspace(data: pd.DataFrame, column: str) -> pd.DataFrame:
  
  if column not in data.columns:
    print("Given column name not in data. Return None.")
    return None
  
  if column != 'omat_tagit':
    data_sub = data[['y_tunnus', column, column + '_refs_lkm']].copy()
  else:
    data_sub = data[['y_tunnus', column]].copy()
  
  data_sub = data_sub.dropna(subset = [column])
  
  if column == 'omat_tagit':
    data_sub = pd.DataFrame(data_sub[column].str.split(';').tolist(), index = data_sub.y_tunnus).stack()
    data_sub = data_sub.reset_index([0, 'y_tunnus'])
    data_sub.columns = ['y_tunnus', column]
  else:
    data_sub[column] = data_sub[column].apply(lambda x: x.split(';'))
    data_sub[column + '_refs_lkm'] = data_sub[column + '_refs_lkm'].apply(lambda x: x.split(';'))
    data_sub = explode(data_sub, [column, column + '_refs_lkm'], fill_value = '')
    
  data_sub[column] = data_sub[column].apply(lambda x: x.strip().lower())
  data_sub['batch'] = pd.Series(timestamp, index = data_sub.index)
  
  return data_sub

In [ ]:
data_liiketoiminta = itewiki_wordspace(data, "liiketoimintaprosessi")
data_liiketoiminta.head()

Out[21]:

,y_tunnus,liiketoimintaprosessi,liiketoimintaprosessi_refs_lkm,batch
0,2454224-2,tuotekehitys ja suunnittelu,0,2021-04-16
1,2707548-3,hr,0,2021-04-16
2,2707548-3,tuotanto,0,2021-04-16
3,2826596-7,asiakaspalvelu,0,2021-04-16
4,2826596-7,toiminnanohjaus erp,0,2021-04-16


In [ ]:
data_erikoisosaaminen = itewiki_wordspace(data, "erikoisosaaminen")
data_erikoisosaaminen.head()

Out[22]:

,y_tunnus,erikoisosaaminen,erikoisosaaminen_refs_lkm,batch
0,2454224-2,mobiilikehitys,0,2021-04-16
1,2454224-2,ohjelmistokehitys,0,2021-04-16
2,2454224-2,webkehitys,0,2021-04-16
3,2707548-3,mobiilikehitys,0,2021-04-16
4,2707548-3,ohjelmistokehitys,0,2021-04-16


In [ ]:
data_teknologia = itewiki_wordspace(data, "teknologia")
data_teknologia.head()

Out[23]:

,y_tunnus,teknologia,teknologia_refs_lkm,batch
0,2454224-2,android,0,2021-04-16
1,2454224-2,open source,0,2021-04-16
2,2454224-2,windows phone,0,2021-04-16
3,2707548-3,android,0,2021-04-16
4,2707548-3,google,0,2021-04-16


In [ ]:
data_toimialakokemus = itewiki_wordspace(data, "toimialakokemus")
data_toimialakokemus.head()

Out[24]:

,y_tunnus,toimialakokemus,toimialakokemus_refs_lkm,batch
0,2454224-2,asiantuntijapalvelut,0,2021-04-16
1,2707548-3,it,0,2021-04-16
2,2826596-7,it,0,2021-04-16
3,2324099-8,asiantuntijapalvelut,0,2021-04-16
4,2324099-8,"hotelli, matkailu ja ravitsemus",0,2021-04-16


In [ ]:
data_tarjonta = itewiki_wordspace(data, "tarjonnan_tyyppi")
data_tarjonta.head()

Out[25]:

,y_tunnus,tarjonnan_tyyppi,tarjonnan_tyyppi_refs_lkm,batch
0,2454224-2,konsultointi,0,2021-04-16
1,2454224-2,valmisohjelmisto,0,2021-04-16
2,2707548-3,toteutustyö,0,2021-04-16
3,2707548-3,vuokratyövoima,0,2021-04-16
4,2826596-7,konsultointi,0,2021-04-16


In [ ]:
data_tagit = itewiki_wordspace(data, "omat_tagit")
data_tagit.head()

Out[26]:

,y_tunnus,omat_tagit,batch
0,2532870-7,tukkukauppa,2021-04-16
1,2532870-7,verkkokaupan logistiikka,2021-04-16
2,2324099-8,tietoturva-auditoinnit,2021-04-16
3,2324099-8,hallinnollinen tietoturva,2021-04-16
4,2324099-8,tietoturva-arviointi,2021-04-16


## Write to Snowflade database

Target database "BF_CLOUDDW_SANDBOX_DEV" and its schema "ITEWIKI".

In [ ]:
user = dbutils.secrets.get("kvdataplatformdatabricks", "snowflakedatabricksuser")
password = dbutils.secrets.get("kvdataplatformdatabricks", "snowflakedatabrickspassword")
 
options = {
  "sfUrl": "xxx.west-europe.azure.snowflakecomputing.com/",
  "sfUser": user,
  "sfPassword": password,
  "sfDatabase": "SANDBOX_DEV",
  "sfSchema": "ITEWIKI",
  "sfWarehouse": "WH_DEVELOPMENT",
  "sfRole": "TESTROLE"
}

In [ ]:
# To convert Pandas DataFrame into Spark DataFrame
# enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [ ]:
# Convert Pandas DataFrame to a Spark DataFrame and write it to Snowflake's dedicated database
def spark_df_to_snowf(data: pd.DataFrame, target_schema_table: str):
  
  # Create a Spark DataFrame from a Pandas DataFrame using Arrow
  spark_data = spark.createDataFrame(data)
  
  spark_data.write \
    .format("snowflake") \
    .options(**options) \
    .option("dbtable", target_schema_table) \
    .mode("append") \
    .save()
  
  return print("Done writing " + target_schema_table)

In [ ]:
if test == False:
  # Very simple data quality check before writing to the database.
  tables = ["itewiki.yrityslista", "itewiki.liiketoimintaprosessi", "itewiki.erikoisosaaminen",
            "itewiki.teknologia", "itewiki.toimialakokemus", "itewiki.tarjonta", "itewiki.tagit"]
  
  if (len(companies)>0 and data.y_tunnus.isnull().sum()== 0):
    output = list(map(lambda x, y: spark_df_to_snowf(x, y), 
                      [data, data_liiketoiminta, data_erikoisosaaminen, 
                       data_teknologia, data_toimialakokemus, data_tarjonta, data_tagit], 
                      tables))
  else:
    print("Did not write data to database. Check data scraper.")

else:
  tables = ["itewiki.temp_yrityslista", "itewiki.temp_liiketoimintaprosessi"]
  output = list(map(lambda x, y: spark_df_to_snowf(x, y), [data, data_liiketoiminta], tables))

Done writing itewiki.yrityslista
Done writing itewiki.liiketoimintaprosessi
Done writing itewiki.erikoisosaaminen
Done writing itewiki.teknologia
Done writing itewiki.toimialakokemus
Done writing itewiki.tarjonta
Done writing itewiki.tagit

In [ ]:
# End